In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()

import sys
import os

COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
MASK_RCNN_MODEL_PATH = 'lib/Mask_RCNN/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
    
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
from lib import utils as siamese_utils
from lib import model as siamese_model
from lib import config as siamese_config
   
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

### Dataset

In [ ]:
# train_classes = coco_nopascal_classes
train_classes = np.array(range(1,81))

In [ ]:
# Load COCO/val dataset
coco_val = siamese_utils.IndexedCocoDataset()
# coco_val.set_active_classes(train_classes)
coco_object = coco_val.load_coco(COCO_DATA, "val", year="2017", return_coco=True)
coco_val.prepare()
coco_val.build_indices()
coco_val.ACTIVE_CLASSES = train_classes

### Model

In [ ]:
class EvalConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1
    NAME = 'coco'
    EXPERIMENT = 'evaluation'
    
config = EvalConfig()
config.display()

### Evaluation

In [ ]:
# Provide training schedule of the model
# When evaluationg intermediate steps the tranining schedule must be provided
train_schedule = OrderedDict()
train_schedule[40] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
train_schedule[120] = {"learning_rate": config.LEARNING_RATE, "layers": "4+"}
train_schedule[160] = {"learning_rate": config.LEARNING_RATE/10, "layers": "all"}

In [ ]:
# Select checkpoint
checkpoints = ['/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/experiments/logs/coco_full_rerun_fixed/siamese_mrcnn_0160.h5']


In [ ]:
# Load and evaluate models
for checkpoint in checkpoints:
    # Create model object in inference mode.
    model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
    model.load_checkpoint(checkpoint, training_schedule=train_schedule)
    # Evaluate only active classes
    active_class_idx = np.array(val_dataset.ACTIVE_CLASSES) - 1
    
    # Evaluate on the validation set
    print('starting evaluation ...')
    siamese_utils.evaluate_dataset(model, coco_val, coco_object, eval_type="bbox", 
                     dataset_type='coco', limit=0, image_ids=None, 
                     class_index=active_class_idx, verbose=1)

### Visualization

In [ ]:
# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
# Select checkpoint
checkpoint = 'checkpoints/siamese_mrcnn_0160.h5'
# Load checkpoint
model.load_checkpoint(checkpoint, training_schedule=train_schedule)

In [ ]:
# Load a random image 
coco_image_ids = [coco_val.image_info[id]["id"] for id in coco_val.image_ids]
image_id = random.choice(range(len(coco_image_ids)))
image = coco_val.load_image(image_id)
print(image_id)

# Load target
category = 1
random_image_id = np.random.choice(coco_val.category_image_index[category])    
# target_image, _, target_class_ids, target_boxes, _ = \
#     modellib.load_image_gt(val_dataset, config, random_image_id, augmentation=False,
#                   use_mini_mask=config.USE_MINI_MASK)

# box_ind = np.random.choice(np.where(target_class_ids == category)[0])   
# tb = target_boxes[box_ind,:]
# target = target_image[tb[0]:tb[2],tb[1]:tb[3],:]
target = siamese_utils.get_one_target(category, coco_val, config)


# Run detection
results = model.detect([target], [image], verbose=1)
r = results[0]
siamese_utils.display_siamese_instances(target, image, r['rois'], r['masks'], r['class_ids'], r['scores'])